# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [36]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from the file `Oders.csv` located in the `data` folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [79]:
# your code here
orders = pd.read_csv('/Users/ayubpathan/Desktop/ironhack/Week2/lab-df-calculation-and-transformation/data/Orders.csv')
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [80]:
orders.sort_values(by='amount_spent',ascending = False).head(20)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
397451,540421,581483,23843,2011,12,5,9,"paper craft , little birdie",80995,2011-12-09 09:15:00,2.08,16446,United Kingdom,168469.60
37126,61619,541431,23166,2011,1,2,10,medium ceramic top storage jar,74215,2011-01-18 10:01:00,1.04,12346,United Kingdom,77183.60
155418,222680,556444,22502,2011,6,5,15,picnic basket wicker 60 pieces,60,2011-06-10 15:28:00,649.50,15098,United Kingdom,38970.00
118352,173382,551697,POST,2011,5,2,13,postage,1,2011-05-03 13:46:00,8142.75,16029,United Kingdom,8142.75
248706,348325,567423,23243,2011,9,2,11,set of tea coffee sugar tins pantry,1412,2011-09-20 11:05:00,5.06,17450,United Kingdom,7144.72
32732,52711,540815,21108,2011,1,2,12,fairy cake flannel assorted colour,3114,2011-01-11 12:55:00,2.10,15749,United Kingdom,6539.40
109624,160546,550461,21108,2011,4,1,13,fairy cake flannel assorted colour,3114,2011-04-18 13:20:00,2.10,15749,United Kingdom,6539.40
308097,421601,573003,23084,2011,10,4,12,rabbit night light,2400,2011-10-27 12:11:00,2.08,14646,Netherlands,4992.00
32730,52709,540815,85123A,2011,1,2,12,white hanging heart t-light holder,1930,2011-01-11 12:55:00,2.55,15749,United Kingdom,4921.50
109620,160542,550461,85123A,2011,4,1,13,white hanging heart t-light holder,1930,2011-04-18 13:20:00,2.40,15749,United Kingdom,4632.00


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [47]:
#orders.sort_values(by='amount_spent', ascending = False).reset_index(drop=False)

In [101]:
# your code here
#cust_grp = orders.groupby('CustomerID').agg({'amount_spent':['max']})

#cust_grp

orders['cust_amount_spent'] = orders['amount_spent'].groupby(orders['CustomerID']).transform('sum')
orders.sort_values(by='amount_spent', ascending = False)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,cust_amount_spent
397451,540421,581483,23843,2011,12,5,9,"paper craft , little birdie",80995,2011-12-09 09:15:00,2.08,16446,United Kingdom,168469.60,168472.50
37126,61619,541431,23166,2011,1,2,10,medium ceramic top storage jar,74215,2011-01-18 10:01:00,1.04,12346,United Kingdom,77183.60,77183.60
155418,222680,556444,22502,2011,6,5,15,picnic basket wicker 60 pieces,60,2011-06-10 15:28:00,649.50,15098,United Kingdom,38970.00,39916.50
118352,173382,551697,POST,2011,5,2,13,postage,1,2011-05-03 13:46:00,8142.75,16029,United Kingdom,8142.75,81024.84
248706,348325,567423,23243,2011,9,2,11,set of tea coffee sugar tins pantry,1412,2011-09-20 11:05:00,5.06,17450,United Kingdom,7144.72,194550.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334242,454463,575579,22437,2011,11,4,11,set of 9 black skull balloons,20,2011-11-10 11:49:00,0.00,13081,United Kingdom,0.00,28337.38
25379,40089,539722,22423,2010,12,2,13,regency cakestand 3 tier,10,2010-12-21 13:45:00,0.00,14911,EIRE,0.00,143825.06
273926,379913,569716,22778,2011,10,4,8,glass cloche small,2,2011-10-06 08:17:00,0.00,15804,United Kingdom,0.00,4206.39
353097,479546,577168,M,2011,11,5,10,manual,1,2011-11-18 10:42:00,0.00,12603,Germany,0.00,739.20


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# your code here


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here
